# Algoritmo para Inferir red booleana

*Para inferir la red booleana se creara un funcion que reciba como parametros la funcion de busqueda, la funcion criterio y la matriz de expresion genica*

## Importacion de Librerias

In [24]:
import numpy as np
from typing import Callable, List, Tuple
from sklearn.metrics import confusion_matrix
import os


## Tipo de Funciones para inferir una red booleana

### Funciones de Busqueda y Evaluacion

In [25]:
# --- Funciones de Búsqueda (Ejemplos) ---

def busqueda_exhaustiva(target_gene_index: int, matriz_expresion: np.ndarray, funcion_criterio: Callable) -> List[int]:
    """
    Busca exhaustivamente los mejores predictores para un gen objetivo, considerando subconjuntos de 1, 2 y 3 genes.

    Args:
        target_gene_index (int): El índice del gen objetivo.
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        funcion_criterio (Callable): La función de criterio para evaluar los predictores.

    Returns:
        List[int]: Una lista con los índices de los mejores predictores encontrados.
    """
    print(f"\nSearching predictors for target gene {target_gene_index}")
    num_genes = matriz_expresion.shape[1]
    mejor_predictores = []
    mejor_criterio = float('inf')  # Inicializamos con un valor alto

    # Iterar sobre los subconjuntos de predictores de tamaño 1, 2 y 3
    for size in range(1, min(4, num_genes)):
      for predictores_indices in _combinations_no_repetir(list(range(num_genes)), size, target_gene_index):
        
        criterio = funcion_criterio(matriz_expresion, target_gene_index, predictores_indices)

        if criterio < mejor_criterio:
            mejor_criterio = criterio
            mejor_predictores = predictores_indices
    for size in range(1, min(4, num_genes)):
        print(f"\nTesting predictor sets of size {size}")
        # ...rest of loop code...

    return mejor_predictores

def _combinations_no_repetir(lista: List, n: int, target_gene_index: int) -> List:
    """
    Genera combinaciones de n elementos de una lista, excluyendo un índice específico.
    """
    if n==1:
        for i in lista:
            if i != target_gene_index:
                yield [i]
        return

    for i in range(len(lista)):
        if lista[i] != target_gene_index:
           for sub_comb in _combinations_no_repetir(lista[i+1:], n - 1, target_gene_index):
              yield [lista[i]]+sub_comb

# --- Funciones de Criterio (Ejemplos) ---

def error_absoluto(matriz_expresion: np.ndarray, target_gene_index: int, predictores_indices: List[int]) -> float:
    """
    Calcula el error absoluto promedio entre la expresión del gen objetivo y la predicción booleana basada en los predictores.
    
    Args:
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        target_gene_index (int): El índice del gen objetivo.
        predictores_indices (List[int]): La lista de índices de los genes predictores.

    Returns:
        float: El error absoluto promedio.
    """
    target_gene = matriz_expresion[:, target_gene_index]
    num_condiciones = len(target_gene)

    if not predictores_indices:
        # En el caso de que no existan predictores devolvemos un valor alto
        return float('inf')

    predicciones = np.zeros(num_condiciones)
    for condicion in range(num_condiciones):
        valores_predictores = [matriz_expresion[condicion, i] for i in predictores_indices] 

        # Implementacion del OR
        prediccion = 0
        if any([True if value > 0 else False for value in valores_predictores]):
          prediccion = 1

        predicciones[condicion] = prediccion


    return np.mean(np.abs(target_gene - predicciones))

### Funciones de Lectura

In [35]:
# --- Funciones de Lectura de Archivo ---

def leer_matriz_deg(archivo_path: str) -> np.ndarray:
    """
   Lee una matriz de expresión de un archivo .deg, donde la primera línea tiene el número de tiempos y el número de genes

    Args:
      archivo_path: La ruta al archivo .deg.

    Returns:
        np.ndarray: La matriz de expresión como un array NumPy.
    """

    if not os.path.exists(archivo_path):
        raise FileNotFoundError(f"El archivo '{archivo_path}' no fue encontrado.")

    try:
        with open(archivo_path, "r") as file:
          lineas = file.readlines()
          if not lineas:
            raise ValueError("El archivo esta vacio.")
          
          header = lineas[0].strip().split()
          if len(header) != 2:
              raise ValueError("La primera línea debe tener dos valores (número de tiempos y número de genes).")
          try:
              num_tiempos = int(header[0])
              num_genes = int(header[1])
          except ValueError:
              raise ValueError("Los valores en la primera línea deben ser números enteros.")

          if num_tiempos <= 0 or num_genes <= 0:
                raise ValueError("Los valores de número de tiempos y número de genes deben ser mayores a 0.")

          matriz = []
          for linea in lineas[1:]:
             linea = linea.strip().split()
             try:
               fila = [int(x) for x in linea]
               if len(fila) != num_genes:
                    raise ValueError(f"La fila {linea} tiene un número incorrecto de columnas.")
               matriz.append(fila)
             except ValueError:
                raise ValueError(f"Error en la conversión de datos en la línea: '{linea}'. Asegúrese de que todos los valores sean enteros.")

          if len(matriz) != num_tiempos:
              raise ValueError(f"El número de filas {len(matriz)} no coincide con el número de instantes de tiempo {num_tiempos}.")
          return np.array(matriz)
    except Exception as e:
       raise Exception(f"Error al leer el archivo {archivo_path}: {e}")



def leer_red_ylu(archivo_path: str) -> dict:
    """Lee la red desde un archivo .ylu."""
    if not os.path.exists(archivo_path):
        raise FileNotFoundError(f"El archivo '{archivo_path}' no fue encontrado.")

    red = {}
    try:
        with open(archivo_path, "r") as file:
            lines = file.readlines()
            for line in lines[1:]: # Saltar la primera linea
               parts = line.strip().split(" ")
               target_gene = int(parts[0])
               if len(parts) > 1:
                  predictors = [int(p) for p in parts[1:]]
                  red[target_gene] = predictors
               else:
                  red[target_gene] = []
    except Exception as e:
       raise Exception(f"Error al leer el archivo {archivo_path}: {e}")
    return red

## Clase para inferir una red booleana

In [32]:


class InferirRedBooleana:
    def __init__(self, funcion_busqueda: Callable, funcion_criterio: Callable, matriz_expresion: np.ndarray):
        """
        Inicializa la clase InferirRedBooleana.

        Args:
            funcion_busqueda (Callable): Una función que busca los mejores predictores.
            funcion_criterio (Callable): Una función que evalúa la calidad de un conjunto de predictores.
            matriz_expresion (np.ndarray): La matriz de expresión génica (genes x condiciones).
        """
        self.funcion_busqueda = funcion_busqueda
        self.funcion_criterio = funcion_criterio
        self.matriz_expresion = matriz_expresion
        self.num_genes = matriz_expresion.shape[1]
        self.red = {}  # Diccionario para almacenar la red inferida: {gen: [predictores]}

    def inferir_red(self) -> dict:
        """
        Infiere la red de regulación génica utilizando las funciones proporcionadas.

        Returns:
            dict: Un diccionario que representa la red inferida (gen: lista_de_predictores).
        """
        for target_gene_index in range(self.num_genes):
            predictores = self.funcion_busqueda(target_gene_index, self.matriz_expresion, self.funcion_criterio)
            self.red[target_gene_index] = predictores
        return self.red

    def obtener_red(self) -> dict:
        """
        Retorna la red inferida.

        Returns:
            dict:  Un diccionario que representa la red inferida (gen: lista_de_predictores).
        """
        return self.red
    def comparar_redes(self, red_verdadera: dict) -> np.ndarray:
        """
        Compara la red inferida con una red "verdadera" y calcula la matriz de confusión.

        Args:
            red_verdadera (dict): La red de referencia (gen: lista_de_predictores).

        Returns:
             np.ndarray: La matriz de confusión.
        """
        y_true = []
        y_pred = []
        for gen_index in range(self.num_genes):
            # Verificación si existen datos en la red verdadera
            if gen_index in red_verdadera:
               for predictor_index in range(self.num_genes):
                    if predictor_index in self.red and gen_index in self.red:
                        y_true.append(1 if predictor_index in red_verdadera[gen_index] else 0)
                        y_pred.append(1 if predictor_index in self.red[gen_index] else 0)

                    elif  predictor_index in self.red:
                        y_true.append(0)
                        y_pred.append(1 if predictor_index in self.red[gen_index] else 0)

                    elif gen_index in self.red:
                        y_true.append(1 if predictor_index in red_verdadera[gen_index] else 0)
                        y_pred.append(0)

        return confusion_matrix(y_true, y_pred)

    def print_frequency_table(self, target_gene: int, predictors: List[int]) -> None:
        """
        Prints the frequency table for a set of predictors and target gene.
        
        Args:
            target_gene (int): Index of target gene
            predictors (List[int]): List of predictor gene indices
        """
        # Initialize frequency table
        freq_table = {
            '0->0': 0, '0->1': 0,
            '1->0': 0, '1->1': 0
        }
        
        # Calculate frequencies
        for t in range(len(self.matriz_expresion)-1):
            # Get predictor state (using OR for multiple predictors)
            pred_state = 0
            for p in predictors:
                if self.matriz_expresion[t,p] == 1:
                    pred_state = 1
                    break
                    
            # Get target state in next timestep
            target_state = self.matriz_expresion[t+1,target_gene]
            
            # Update frequency table
            key = f"{pred_state}->{target_state}"
            freq_table[key] += 1
            
        print(f"\nFrequency Table for Target Gene {target_gene}")
        print(f"Predictors: {predictors}")
        print("---------------")
        print("Pred -> Target | Count")
        print("---------------")
        for k,v in freq_table.items():
            print(f"{k:<12} | {v:>5}")
        print("---------------\n")
    


## Ejemplo de Uso

In [34]:
# --- Ejemplo de uso ---

if __name__ == '__main__':
   try:
      # Ruta al archivo .deg
      archivo_deg_path = "expresion_genica.deg"

      # Crear un archivo de ejemplo si no existe
      if not os.path.exists(archivo_deg_path):
        with open(archivo_deg_path, "w") as f:
            f.write("1 0 1 0\n")
            f.write("0 1 1 0\n")
            f.write("1 1 0 1\n")
            f.write("0 0 1 1\n")
            f.write("1 0 0 1\n")

      # Leer la matriz de expresión del archivo
      matriz_exp = leer_matriz_deg(archivo_deg_path)

      # Red "verdadera" de ejemplo
      red_verdadera = {
        0: [1, 2],  # El gen 0 es regulado por los genes 1 y 2
        1: [0, 2],  # El gen 1 es regulado por los genes 0 y 2
        2: [3],     # El gen 2 es regulado por el gen 3
      }

      # Crear una instancia de InferirRedBooleana
      red_inferencia = InferirRedBooleana(busqueda_exhaustiva, error_absoluto, matriz_exp)

      # Infiere la red
      red_inferida = red_inferencia.inferir_red()

      # Imprimir la red inferida
      print("Red Inferida:", red_inferida)

      # Comparar las redes y obtener la matriz de confusión
      matriz_confusion = red_inferencia.comparar_redes(red_verdadera)

      # Imprimir la matriz de confusión
      print("Matriz de Confusión:")
      print(matriz_confusion)
      red_inferencia = InferirRedBooleana(busqueda_exhaustiva, error_absoluto, matriz_exp)
        
      # Test with specific target gene
      target_gene = 0
      predictors = red_inferencia.funcion_busqueda(target_gene, matriz_exp, error_absoluto)
      red_inferencia.print_frequency_table(target_gene, predictors)
        
      # Continue with full network inference
      red_inferida = red_inferencia.inferir_red()
      print("Red Inferida:", red_inferida)
        
   except Exception as e:
      print(f"Se produjo un error inesperado: {e}")
 


Searching predictors for target gene 0

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searching predictors for target gene 1

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searching predictors for target gene 2

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searching predictors for target gene 3

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searching predictors for target gene 4

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searching predictors for target gene 5

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searching predictors for target gene 6

Testing predictor sets of size 1

Testing predictor sets of size 2

Testing predictor sets of size 3

Searc